In [1]:
import spacy

nlp = spacy.load("en_core_web_lg")

In [2]:
import pandas as pd

# query = "Query_NYT_5y_surveillance"
query = "Query_AI-magazines-mediacloud-2021-24"
# query = "Query_US_5y_Svalbard"

CSV_FILE = query + ".csv"
FEATHER_FILE = query + ".feather"

df = pd.read_feather(FEATHER_FILE)

df.sample(3)

,id,indexed_date,language,media_name,media_url,publish_date,title,url,year,text,text_downloaded
1137,0b286ed4a61b2e934c46b85613d431a4001be15bea20be...,2024-02-24 11:20:05.775846,en,technologyreview.com,technologyreview.com,2023-07-18,Meta’s latest AI model is free for all,https://www.technologyreview.com/2023/07/18/10...,2023,None,False
1322,2132006236843ff206a7f5b9101e298c3e568c4ca2c8da...,2024-02-28 07:14:47.832928,en,technologyreview.com,technologyreview.com,2023-05-12,The open-source AI boom is built on Big Tech’s...,https://www.technologyreview.com/2023/05/12/10...,2023,The open-source AI boom is built on Big Tech’s...,True
232,f852289664ca2dffb85c56ff4dc4e32633a60df9c58e2c...,2024-06-22 12:17:19.325637,en,wired.com,wired.com,2024-06-22,Everything’s About to Get a Hell of a Lot More...,https://www.wired.com/story/everythings-about-...,2024,If you buy something using links in our storie...,True


In [4]:
df['text'] = df['text'].fillna('')

In [5]:
from tqdm.notebook import tqdm

GPE = [""] * len(df)
IND = [""] * len(df)
ORG = [""] * len(df)

# Browse the data ...
for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row.text
    doc = nlp(text)

    entities = {
        "GPE": [],
        "IND": [],
        "ORG": [],
    }

    for ent in (e for e in doc.ents if e.label_ == "GPE"):
        entities["GPE"].append(ent.text)

    for ent in (e for e in doc.ents if e.label_ == "PERSON"):
        entities["IND"].append(ent.text)

    for ent in (e for e in doc.ents if e.label_ == "ORG"):
        entities["ORG"].append(ent.text)

    GPE[index] = entities["GPE"]
    IND[index] = entities["IND"]
    ORG[index] = entities["ORG"]

  0%|          | 0/2633 [00:00<?, ?it/s]

In [6]:
df['GPE'] = GPE
df['IND'] = IND
df['ORG'] = ORG

df.to_feather(FEATHER_FILE)